In [1]:
import pprint as pp

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
import keras
from keras import layers
from keras.losses import BinaryCrossentropy


2023-09-04 19:14:22.177803: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 19:14:22.206153: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-04 19:14:22.207092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 19:14:22.833672: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the Titanic dataset.
titanic_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic = titanic_df.astype({
    'survived': 'int8',
    'age': 'int8',
    'n_siblings_spouses': 'int8',
    'parch': 'int8',
    'fare': 'int16',
})

titanic.info()

titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   survived            627 non-null    int8  
 1   sex                 627 non-null    object
 2   age                 627 non-null    int8  
 3   n_siblings_spouses  627 non-null    int8  
 4   parch               627 non-null    int8  
 5   fare                627 non-null    int16 
 6   class               627 non-null    object
 7   deck                627 non-null    object
 8   embark_town         627 non-null    object
 9   alone               627 non-null    object
dtypes: int16(1), int8(4), object(5)
memory usage: 28.3+ KB


In [3]:

# attempts to (eventually) preprocess a dataframe of features
# returns all of the collected inputs and preprocessed features 
def process_dataframe_inputs(feature_df):

  # first we walk through each feature in the dataframe, make an input for it,
  # normalize it using the feature_df data, then collect it into lists of
  # inputs and preprocessed inputs, respectively

  input_layers, preprocessed_inputs  = [], []
  for name, col in feature_df.items():

    # TODO do this with log() ... print("processing %s ..." % name)

    # convert to tensorflow types. this is straightforward except 
    # for objects, treat them as strings.
    dtype_str = str(col.dtype) if col.dtype != object else "string"
    cur_dtype = tf.as_dtype(dtype_str)
    cur_input = tf.keras.Input(shape=(1,), name=name, dtype=cur_dtype)
    input_layers.append(cur_input)  

    if cur_dtype == tf.string:
      lookup = layers.StringLookup(vocabulary=np.unique(col))    
      norm = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())
      normalized_input = norm(lookup(cur_input))
      preprocessed_inputs.append(normalized_input)

    else:
      norm = layers.Normalization(axis=None)
      norm.adapt(col)
      normalized_input = norm(cur_input)
      preprocessed_inputs.append( normalized_input )

  # prepare the return values by concatentating the preprocessed inputs
  # and creating a model for which they serve as outputs.
  # 
  # returns the input layers and the processed_inputs, respectively

  preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)
  preprocessing_model = tf.keras.Model(input_layers, preprocessed_inputs_cat)
  preprocessed_inputs = preprocessing_model(input_layers)

  return input_layers, preprocessed_inputs


In [4]:

input_layers, preprocessed_inputs = process_dataframe_inputs(titanic_features)

seqential_hidden_model = tf.keras.Sequential([
  layers.Dropout(0.10),
  layers.Dense(16),
  layers.Dense(16),
  layers.Dense(8),
  layers.Dense(1)
])
result = seqential_hidden_model(preprocessed_inputs)
titanic_model = tf.keras.Model(input_layers, result)
titanic_model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

titanic_features_dict = {
  name: np.array(value) for name, value in titanic_features.items()
}

hist = titanic_model.fit(x=titanic_features_dict, y=titanic_labels, epochs=100, batch_size=25)


2023-09-04 19:14:44.968521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100
26/26 [==============================] - 1s 3ms/step - loss: 5.9552 - accuracy: 0.4242
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 5.8530 - accuracy: 0.4737
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 5.5888 - accuracy: 0.5295
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 5.1668 - accuracy: 0.5407
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 5.1112 - accuracy: 0.5726
Epoch 6/100
26/26 [==============================] - 0s 2ms/step - loss: 5.1041 - accuracy: 0.5439
Epoch 7/100
26/26 [==============================] - 0s 3ms/step - loss: 4.8088 - accuracy: 0.5742
Epoch 8/100
26/26 [==============================] - 0s 2ms/step - loss: 4.3630 - accuracy: 0.5694
Epoch 9/100
26/26 [==============================] - 0s 5ms/step - loss: 3.6002 - accuracy: 0.5662
Epoch 10/100
26/26 [==============================] - 0s 2ms/step - loss: 2.8844 - accuracy: 0.6124
Epoch 11/

In [ ]:

fig, ax1 = plt.subplots()
ax1.plot(hist.epoch, hist.history['loss'], label='loss', color="blue")
ax2 = ax1.twinx()
ax2.plot(hist.epoch, hist.history['accuracy'], label='accuracy', color="red")

plt.show()

# tf.keras.utils.plot_model(model = titanic_model, rankdir="LR",)

